In [1]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
import csv
from sqlalchemy import create_engine
from datetime import datetime
import locale

In [2]:
# Estabelecimento de conexão com o banco de dados
# para extração das informações das tabelas necessárias

database_config = {
    "host": "cat42.cohe0l2miaal.us-east-1.rds.amazonaws.com",
    "user": "master",
    "password": "dms191411",
    "database": "cat"
}

# Create a connection to the MySQL database
engine = create_engine(f"mysql+pymysql://{database_config['user']}:{database_config['password']}@{database_config['host']}/{database_config['database']}")

In [3]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha = pd.read_csv('Loja_44/tabela_2_loja_44_version_final_v8.csv', dtype=str)
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')
template = pd.read_excel('CFOP_e_template.xlsx', sheet_name='Template 150', dtype=str)
planilha['DESCRICAO'] = planilha['DESCRICAO'].replace('"', "'")
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050.0,18.0,NaN
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455.0,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,18.0,1300301.0
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050.0,18.0,NaN
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110.0,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,18.0,1300101.0
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050.0,18.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,27.2947078125,35200345453214001042550180007353931100320204,2,45453214001042.0,"FLENUS 22,25MG 20'S ARESE",7896317906226.0,UN,30049099.0,18.0,1300101.0
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,13.64735390625,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099.0,18.0,NaN
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,0.0,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099.0,18.0,NaN
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,27.244800000000005,35200744463156000184550840006132201109539626,4,44463156000184.0,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099.0,18.0,1300401.0


In [4]:
planilha.head()

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050.0,18.0,NaN
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455.0,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,18.0,1300301.0
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050.0,18.0,NaN
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110.0,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,18.0,1300101.0
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050.0,18.0,NaN


In [5]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].str.replace('.', ',000')

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

C:\Users\vinic\AppData\Local\Temp\ipykernel_12368\2782868166.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  planilha['QTD_CAT'] = planilha['QTD_CAT'].str.replace('.', ',000')


,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,18.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,27.2947078125,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226.0,UN,30049099,18.0,1300101
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,13.64735390625,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,0.0,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,27.244800000000005,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401


In [6]:
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,18.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,27.2947078125,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226.0,UN,30049099,18.0,1300101
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,13.64735390625,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,0.0,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,27.244800000000005,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401


In [7]:
query = 'SELECT * FROM tabela_efds_modelo55 WHERE LOJA = 44'

efd = pd.read_sql(query, engine)
efd

,ID,LOJA,DATA_EFD,DATA_ENTRADA,DATA_SAIDA,CHAVE_ORIGINAL,COD_DO_ITEM,CODPROF,CFOP,DESC_PROD,QUANTI_EFDS,EAN,NCM,FATOR,ICMS,CEST
0,1159617,44,01-2020,09/01/2020,15/01/2020,35200171605265000242550020000052631164184284,1,184812,1409,Brintellix 10mg 60'S C1,1,,30049069,1,,
1,1159618,44,01-2020,27/01/2020,27/01/2020,35200171605265000404550020000039561165263676,1,750547,1409,LEXAPRO 10MG 28'S LUNDBECK C1,1,5702157150809,30049059,1,18,13.001.00
2,1159619,44,01-2020,13/01/2020,13/01/2020,35200171605265000595550020000038721164385051,1,176883,1202,Cartidres 30'S MCG,2,7898623352575,21069030,1,,
3,1159620,44,01-2020,30/01/2020,30/01/2020,35200171605265004159550020000001871165527919,1,895755,1409,GALVUS MET 50/1000MG 56'S NOV,1,7896261016262,30049069,1,18,13.001.00
4,1159621,44,01-2020,15/01/2020,21/01/2020,35200171605265004230550020000001161164563671,1,126532,1409,"EXFORGE HCT 160/12,5/5MG 28'S",1,7896261015586,30049069,1,18,13.001.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144947,1304564,44,12-2020,03/12/2020,03/12/2020,35201271605265000595550020000046051183194751,1,141028,1411,DROSPIREN.+ETINIL.3MG 63'S LEG,1,,30066000,1,12,13.002.00
144948,1304565,44,12-2020,06/12/2020,06/12/2020,35201271605265000595550020000046091183312613,1,43570,1411,SERINGA BD 3ML AG30X7 UNIDADE,400,,90183219,100,12,13.015.00
144949,1304566,44,12-2020,01/12/2020,01/12/2020,35201271605265000595550020000045981182969192,1,150284,1411,LEITE NAN 1 SUPREME 800G,1,,19011010,1,18/18,17.014.00
144950,1304567,44,12-2020,05/12/2020,05/12/2020,35201271605265000595550020000046081183309097,1,213728,1411,Biod Sensibio Ar 40ml,1,,34012010,1,18,20.036.00


In [8]:
planilha = planilha.drop_duplicates()
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550.0,UN,30045050,18.0,nan
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,18.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,27.2947078125,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226.0,UN,30049099,18.0,1300101
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,13.64735390625,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,0.0,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226.0,UN,30049099,18.0,nan
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,27.244800000000005,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401


In [9]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,SALDO_FINAL_MES_ICMS,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,NaN,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,nan
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35.4528,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,nan
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,NaN,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,NaN,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,18.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,27.2947078125,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226,UN,30049099,18.0,1300101
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,13.64735390625,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,nan
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,0.0,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,nan
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,27.244800000000005,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401


In [10]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['102019', '112019', '122019', '022020', '032020', '042020',
       '052020', '062020', '072020', '082020', '102020', '112020',
       '122020', '012020', '092020'], dtype=object)

In [11]:
planilha.replace('nan', np.nan, inplace=True)

In [12]:
planilha['DATA'].astype(str).iloc[0][8:]

'01'

In [13]:
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST,Data
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,102019
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301,102019
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,112019
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101,112019
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,18.0,NaN,112019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226,UN,30049099,18.0,1300101,032020
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,052020
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,062020
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401,072020


In [14]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0             NaN
1         1300301
2             NaN
3         1300101
4             NaN
           ...   
379070    1300101
379071        NaN
379072        NaN
379073    1300401
379074        NaN
Name: CEST, Length: 379075, dtype: object

In [15]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST,Data
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,102019
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301,102019
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,112019
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101,112019
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,18.0,NaN,112019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226,UN,30049099,18.0,1300101,032020
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,052020
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,062020
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401,072020


In [16]:
planilha.columns

Index(['Unnamed: 0', 'DATA', 'CFOP', 'COD_ITEM', 'IND_OPER', 'COD_LEGAL',
       'SUB_TIPO', 'QTD_CAT', 'SALDO_INI_QTD', 'SALDO_INI_ICMS',
       'QTD_ent1_devolv_ent', 'ICMS_TOT', 'ICMS_TOT_ent_unit',
       'ULT_ICMS_TOT_ent_unit', 'ICMS_TOT_1', 'qtd_saida_1_devolv_saida',
       'QTD_SALDO', 'ICMS_SAIDA_UNI', 'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA',
       'ICMS_TOT_SALDO', 'ICMS_TOT_PCAT', 'VLR_CONF_0', 'VLR_CONFR_UNIT',
       'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'VLR_RESSARCIMENTO',
       'VLR_COMPLEMENTO', 'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'REF',
       'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS', 'CHV_DOC', 'NUM_ITEM',
       'CNPJ EMITENTE', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M',
       'ALIQUOTA', 'CEST', 'Data'],
      dtype='object')

In [17]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST,Data
29,374,2020-05-05,1411,100073,0,NaN,-1,1,NaN,NaN,...,35200571605265000595550020000040681170777724,1,71605265000595,Os Cal D 60S Sanofi/Aventis,SEM GTIN,UN,30045050,18.0,1300301,052020
353,392,2020-05-11,1411,100693,0,NaN,-1,1,NaN,NaN,...,35200571605265000595550020000040871171122824,1,71605265000595,Montelair 10mg 30S,SEM GTIN,UN,30049079,7.0,1300300,052020
1190,268,2020-03-06,1411,101113,0,NaN,-1,1,NaN,NaN,...,35200371605265000595550020000039501167484722,1,71605265000595,Clor Naratriptana 10S Novaqui,SEM GTIN,UN,30049099,12.0,1300200,032020
1191,269,2020-03-06,1411,101113,0,NaN,-1,1,NaN,NaN,...,35200371605265000595550020000039501167484722,2,71605265000595,Clor Naratriptana 10S Novaqui,SEM GTIN,UN,30049099,12.0,1300200,032020
2235,109,2019-11-23,1411,101261,0,NaN,-1,1,NaN,NaN,...,35191171605265000595550020000037871161292000,1,71605265000595,Sh Tresemme 400ml Cacho Perfei,SEM GTIN,UN,33051000,18.0,2801100,112019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378310,317,2020-03-31,1411,98970,0,NaN,-1,1,NaN,NaN,...,35200371605265000595550020000040091168956600,1,71605265000595,Cond Tresemme 400ml Liso E Sed,SEM GTIN,UN,33059000,25.0,2801400,032020
378881,623,2020-08-25,1411,99036,0,NaN,-1,1,NaN,NaN,...,35200871605265000595550020000043821177370464,2,71605265000595,Ciprofibrato 100mg 30S Nq,SEM GTIN,UN,30049099,12.0,1300200,082020
378882,624,2020-08-25,1411,99036,0,NaN,-1,1,NaN,NaN,...,35200871605265000595550020000043821177370464,3,71605265000595,Ciprofibrato 100mg 30S Nq,SEM GTIN,UN,30049099,12.0,1300200,082020
378883,625,2020-08-25,1411,99036,0,NaN,-1,1,NaN,NaN,...,35200871605265000595550020000043821177370464,4,71605265000595,Ciprofibrato 100mg 30S Nq,SEM GTIN,UN,30049099,12.0,1300200,082020


In [18]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,Unnamed: 0,DATA,CFOP,COD_ITEM,IND_OPER,COD_LEGAL,SUB_TIPO,QTD_CAT,SALDO_INI_QTD,SALDO_INI_ICMS,...,CHV_DOC,NUM_ITEM,CNPJ EMITENTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,ALIQUOTA,CEST,Data
0,197762,2019-10-01,5405,100073,1,1.0,1,1,3.0,35.4726,...,35191071605265000595590003114260972468062852,1,NaN,1971 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,102019
1,1854,2019-10-03,1403,100073,0,NaN,1,1,NaN,NaN,...,35191005651966000455550240004004761539366521,8,5651966000455,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300301,102019
2,305534,2019-11-01,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261029698537006,1,NaN,5139 - OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,NaN,112019
3,11949,2019-11-06,1403,100073,0,NaN,1,1,NaN,NaN,...,35191127849963000110550010006031801006012824,17,27849963000110,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,18.0,1300101,112019
4,316600,2019-11-21,5405,100073,1,1.0,1,1,NaN,NaN,...,35191171605265000595590003114261058425299890,1,NaN,8353 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,18.0,NaN,112019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379070,54999,2020-03-11,1403,99671,0,NaN,1,1,NaN,NaN,...,35200345453214001042550180007353931100320204,2,45453214001042,"FLENUS 22,25MG 20'S ARESE",7896317906226,UN,30049099,18.0,1300101,032020
379071,247963,2020-05-25,5405,99671,1,1.0,1,1,NaN,NaN,...,35200571605265000595590003114261366257626039,1,NaN,"3763 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,052020
379072,350612,2020-06-13,5405,99671,1,1.0,1,1,NaN,NaN,...,35200671605265000595590003114261397752679870,1,NaN,"9667 - FLENUS 22,25MG 20S ARESE",7896317906226,UN,30049099,18.0,NaN,062020
379073,89471,2020-07-15,1403,99671,0,NaN,1,2,NaN,NaN,...,35200744463156000184550840006132201109539626,4,44463156000184,"FLENUS 22,25MG 20'S ARESE",NaN,CX,30049099,18.0,1300401,072020


In [19]:
list(planilha['CODIGO_BARRA'].unique())

['7891058011550',
 '7891058011550 ',
 nan,
 '7890282800688',
 '7891268148961',
 '7891024139325',
 '7890282800701',
 '7892828001405',
 '7891024114865',
 '7891024114865 ',
 '7891058325039',
 '7891058325039 ',
 '7891058326005',
 '7896658014215',
 '7897930762695',
 '7897322705415',
 '7891150018761',
 '7891150006058',
 '7891150006072',
 '7891317462031',
 '7891317462031 ',
 '7899304803737',
 '7899304803713',
 '4260085521425',
 '5702191025644',
 '7895296105040',
 '7898505091479',
 '7898505091479 ',
 '7896181901372',
 '7899304803706',
 '7899304803744',
 '7899304803720',
 '7891150018853',
 '7896382706240',
 '7896026304306',
 '7891058260071',
 '7891058263096 ',
 '7891058263096',
 '7899026457645',
 '7899026457645 ',
 '7899026459960',
 '7899026459908',
 '7899026457621',
 '7899026459649',
 '7899026459748',
 '7899026458949',
 '7897337706711',
 '7897322709529',
 '7896026304276',
 '7896026304337',
 '7891142144201',
 '7891010973902',
 '7891010974312',
 '7896004728834',
 '7891058489038',
 '7891058000165

In [20]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'COD_ITEM', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191071605265000595590003114260972468062852,2019-10-01,01102019,1,1,100073,5405,1,NaN,0.0,0
1,1100,35191005651966000455550240004004761539366521,2019-10-03,03102019,8,0,100073,1403,1,11.8044,NaN,NaN
2,1100,35191171605265000595590003114261029698537006,2019-11-01,01112019,1,1,100073,5405,1,NaN,0.0,0
3,1100,35191127849963000110550010006031801006012824,2019-11-06,06112019,17,0,100073,1403,1,11.8242,NaN,NaN
4,1100,35191171605265000595590003114261058425299890,2019-11-21,21112019,1,1,100073,5405,1,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
379070,1100,35200345453214001042550180007353931100320204,2020-03-11,11032020,2,0,99671,1403,1,13.752,NaN,NaN
379071,1100,35200571605265000595590003114261366257626039,2020-05-25,25052020,1,1,99671,5405,1,NaN,0.0,0
379072,1100,35200671605265000595590003114261397752679870,2020-06-13,13062020,1,1,99671,5405,1,NaN,0.0,0
379073,1100,35200744463156000184550840006132201109539626,2020-07-15,15072020,4,0,99671,1403,2,27.244800000000005,NaN,NaN


In [21]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0         102019
1         102019
2         112019
3         112019
4         112019
           ...  
379070    032020
379071    052020
379072    062020
379073    072020
379074    092020
Name: Data, Length: 379075, dtype: object

In [22]:
def transform_qtd(value):
    if pd.notnull(value):
        return str(value) + ',000'
    else:
        return value

# Apply the transformation function to the 'FATOR' column
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].astype(str).apply(transform_qtd)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191071605265000595590003114260972468062852,102019,01102019,1,1,100073,5405,"1,000",NaN,0.0,0
1,1100,35191005651966000455550240004004761539366521,102019,03102019,8,0,100073,1403,"1,000",11.8044,NaN,NaN
2,1100,35191171605265000595590003114261029698537006,112019,01112019,1,1,100073,5405,"1,000",NaN,0.0,0
3,1100,35191127849963000110550010006031801006012824,112019,06112019,17,0,100073,1403,"1,000",11.8242,NaN,NaN
4,1100,35191171605265000595590003114261058425299890,112019,21112019,1,1,100073,5405,"1,000",NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
379070,1100,35200345453214001042550180007353931100320204,032020,11032020,2,0,99671,1403,"1,000",13.752,NaN,NaN
379071,1100,35200571605265000595590003114261366257626039,052020,25052020,1,1,99671,5405,"1,000",NaN,0.0,0
379072,1100,35200671605265000595590003114261397752679870,062020,13062020,1,1,99671,5405,"1,000",NaN,0.0,0
379073,1100,35200744463156000184550840006132201109539626,072020,15072020,4,0,99671,1403,"2,000",27.244800000000005,NaN,NaN


In [23]:
df_1100.dtypes

COD_REG           object
CHV_DOC           object
Data              object
DATA              object
NUM_ITEM          object
IND_OPER          object
COD_ITEM          object
CFOP              object
QTD_FIN           object
ICMS_TOT_PCAT     object
VLR_CONFR_PCAT    object
COD_LEGAL_PCAT    object
dtype: object

In [24]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191071605265000595590003114260972468062852,102019,01102019,1,1,100073,5405,"1,000",NaN,0.0,0
1,1100,35191005651966000455550240004004761539366521,102019,03102019,8,0,100073,1403,"1,000",11.8044,NaN,NaN
2,1100,35191171605265000595590003114261029698537006,112019,01112019,1,1,100073,5405,"1,000",NaN,0.0,0
3,1100,35191127849963000110550010006031801006012824,112019,06112019,17,0,100073,1403,"1,000",11.8242,NaN,NaN
4,1100,35191171605265000595590003114261058425299890,112019,21112019,1,1,100073,5405,"1,000",NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
379070,1100,35200345453214001042550180007353931100320204,032020,11032020,2,0,99671,1403,"1,000",13.752,NaN,NaN
379071,1100,35200571605265000595590003114261366257626039,052020,25052020,1,1,99671,5405,"1,000",NaN,0.0,0
379072,1100,35200671605265000595590003114261397752679870,062020,13062020,1,1,99671,5405,"1,000",NaN,0.0,0
379073,1100,35200744463156000184550840006132201109539626,072020,15072020,4,0,99671,1403,"2,000",27.244800000000005,NaN,NaN


In [25]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_csv('Loja_44/1050_loja_44_final_v6.csv', dtype=str)

In [26]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = mes_ano
arquivo_1050

,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,QTD_FIM,ICMS_FIN,DATA
0,0,2019-10,100073,3.0,35.4726,3.0,35.4528,102019
1,6681,2019-11,100073,3.0,35.4528,3.0,35.468199999999996,112019
2,13362,2019-12,100073,3.0,35.468199999999996,3.0,35.4132,122019
3,20043,2020-01,100073,3.0,35.4132,3.0,35.4132,012020
4,26724,2020-02,100073,3.0,35.4132,3.0,35.4264,022020
...,...,...,...,...,...,...,...,...
100210,73490,2020-08,99671,2.0,27.244800000000005,2.0,27.244800000000005,082020
100211,80171,2020-09,99671,2.0,27.244800000000005,1.0,13.622400000000003,092020
100212,86852,2020-10,99671,1.0,13.622400000000003,1.0,13.622400000000003,102020
100213,93533,2020-11,99671,1.0,13.622400000000003,1.0,13.622400000000003,112020


In [27]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
022020
032020
042020
052020
062020
072020
082020
102020
112020
122020
012020
092020


In [47]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='utf-8') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [29]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [30]:
dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']

# Initialize an empty list to store DataFrames for each file
dfs = []

# Iterate through each file in the folder
for date in dates:
    
    read_files = glob.glob(f"Loja_44/Loja 44/{date}.txt")
    for f in read_files:
        # Read the .txt file
        with open(f,  'r', encoding='latin-1') as file:
            lines = file.readlines()
        
        # Filter rows starting with '|0150|' and split them into columns
        filtered_lines = [line.strip() for line in lines if line.startswith('|0150|')]
        data = [line.split('|') for line in filtered_lines]
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=['ID','REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                                'IE', 'COD_MUN', 'Column1', 'RUA', 'NUMERO', 'Column2', 'BAIRRO'', Column3', 'Column4'])
       

        df['PERIODO'] = [f'{date}']*df.shape[0]
        df = df[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'COD_MUN', 'IE', 'PERIODO']]
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df_0150 = pd.concat(dfs, ignore_index=True)
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,CLI000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,,102019
1,0150,CLI000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,CLI000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,CLI000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
4,0150,FOR000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
...,...,...,...,...,...,...,...,...,...
860,0150,FOR753010428,GILBERTO JACINTHO LUCIANO,1058,,11973070871,3548708,,122020
861,0150,FOR753010663,ARLETE BECHIATO CAPOLETTO VERTAMATTI,1058,,67021417853,3548708,,122020
862,0150,FOR753012516,CARLOS ROGERIO DA SILVA,1058,,27575311823,3548708,,122020
863,0150,FOR753012691,MADALENA SALINAS DA SILVA,1058,,80658075853,3548708,,122020


In [31]:
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['COD_PART'] = df_0150['COD_PART'].str[3:]
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,NaN,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
4,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
...,...,...,...,...,...,...,...,...,...
860,0150,753010428,GILBERTO JACINTHO LUCIANO,1058,NaN,11973070871,3548708,NaN,122020
861,0150,753010663,ARLETE BECHIATO CAPOLETTO VERTAMATTI,1058,NaN,67021417853,3548708,NaN,122020
862,0150,753012516,CARLOS ROGERIO DA SILVA,1058,NaN,27575311823,3548708,NaN,122020
863,0150,753012691,MADALENA SALINAS DA SILVA,1058,NaN,80658075853,3548708,NaN,122020


In [32]:
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,NaN,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
5,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,102019
...,...,...,...,...,...,...,...,...,...
834,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,,3549904,645333991113,122020
837,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,,3508504,234086958110,122020
838,0150,000359053,RESTAURANTE E LANCHONETE D JULIA ME,1058,17906239000128,,3548708,635684676111,122020
842,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,,3505708,206165226110,122020


In [33]:
mask_duplicates = df_0150[df_0150['PERIODO'] == '112019'].duplicated(subset='CNPJ', keep='first')
# Invert the mask to keep the first occurrence of each unique 'CNPJ' value
mask_to_keep = ~mask_duplicates

# Apply the mask to the DataFrame to keep the desired rows for the current 'PERIODO'
filtered_df = df_0150[(df_0150['PERIODO'] == '112019') | mask_to_keep]
filtered_df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
48,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,112019
49,0150,000000455,Master formula farmacia de manip. ltda-me,1058,71605265000595,,3548708,635726183115,112019
50,0150,000000496,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004663,,3550308,126790472110,112019
51,0150,000000498,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004825,,3550308,128110725115,112019
54,0150,751872165,Conde atacadista LTDA,1058,27849963000110,,3508504,234086958110,112019
57,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,112019
58,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,112019
59,0150,000002004,DHV Distribuidora Hospitalar do Vale Ltda - EPP,1058,23276873000127,,3549904,645739349118,112019
60,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,,3543402,639117330119,112019
61,0150,000002016,Servimed comercial ltda,1058,44463156000184,,3506003,209017636112,112019


In [34]:
filtered_df['CNPJ'].unique()

array(['71605265000404', '71605265000595', '71605265004663',
       '71605265004825', '27849963000110', '71605265004582',
       '61940292000137', '23276873000127', '05651966000455',
       '44463156000184', '01206820000520', '45453214001042',
       '97554090000102', '10328983000143', '17906239000128',
       '07259917000154'], dtype=object)

In [35]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['PERIODO'].unique()

# Iterate through unique 'PERIODO' values
for periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['PERIODO'] == periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\2729301529.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,NaN,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
4,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,102019
...,...,...,...,...,...,...,...,...,...
272,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,,3548906,637117955110,122020
273,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,,3549904,645333991113,122020
274,0150,000359053,RESTAURANTE E LANCHONETE D JULIA ME,1058,17906239000128,,3548708,635684676111,122020
275,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,,3505708,206165226110,122020


In [36]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['PERIODO'] == '112019']

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
16,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,112019
17,0150,000000455,Master formula farmacia de manip. ltda-me,1058,71605265000595,,3548708,635726183115,112019
18,0150,000000496,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004663,,3550308,126790472110,112019
19,0150,000000498,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004825,,3550308,128110725115,112019
20,0150,751872165,Conde atacadista LTDA,1058,27849963000110,,3508504,234086958110,112019
21,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,112019
22,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,112019
23,0150,000002004,DHV Distribuidora Hospitalar do Vale Ltda - EPP,1058,23276873000127,,3549904,645739349118,112019
24,0150,000002006,D.Center Distribuidora Ltda.,1058,05651966000455,,3543402,639117330119,112019
25,0150,000002016,Servimed comercial ltda,1058,44463156000184,,3506003,209017636112,112019


In [37]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for cnpj in df_0150_final[df_0150_final['PERIODO'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['PERIODO'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['PERIODO'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
            
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO


In [38]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for ie in df_0150_final[df_0150_final['PERIODO'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['PERIODO'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['PERIODO'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])
            
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO


In [39]:
dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']

# Initialize an empty list to store DataFrames for each file
dfs = []

# Iterate through each file in the folder
for date in dates:
    
    read_files = glob.glob(f"Loja_44/Loja 44/{date}.txt")
    for f in read_files:
        # Read the .txt file
        with open(f,  'r', encoding='latin-1') as file:
            lines = file.readlines()
        
        # Filter rows starting with '|0150|' and split them into columns
        filtered_lines = [line.strip() for line in lines if line.startswith('|0000|')]
        data = [line.split('|') for line in filtered_lines]
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=['ID','REG', 'Col2','Col3', 'INI', 'FIM', 'NOME', 'CNPJ',
                                'CPF', 'ESTADO', 'IE', 'COD_MUN', 'Col4', 'Col8', 'Col5','Col6', 'Col7'])
       

        df['PERIODO'] = [f'{date}']*df.shape[0]
        df['COD_VER'] = ['01'] * df.shape[0]
        df['COD_FIN'] = ['00'] * df.shape[0]
        df = df[['REG', 'PERIODO', 'NOME', 'CNPJ', 'IE',
                'COD_MUN', 'COD_VER', 'COD_FIN']]
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df_0000 = pd.concat(dfs, ignore_index=True)
df_0000

,REG,PERIODO,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,0000,102019,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
1,0000,112019,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
2,0000,122019,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
3,0000,012020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
4,0000,022020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
5,0000,032020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
6,0000,042020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
7,0000,052020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
8,0000,062020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00
9,0000,072020,Master Formula Farmacia de Manipulacao LTDA,71605265000595,635726183115,3548708,01,00


In [40]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
022020
032020
042020
052020
062020
072020
082020
102020
112020
122020
012020
092020


In [41]:
df_0150['CNPJ'] = df_0150['CNPJ'].astype(str).replace('', np.nan)
df_0150['IE'] = df_0150['IE'].astype(str).replace('', np.nan)
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,nan,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
5,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,102019
...,...,...,...,...,...,...,...,...,...
834,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,,3549904,645333991113,122020
837,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,,3508504,234086958110,122020
838,0150,000359053,RESTAURANTE E LANCHONETE D JULIA ME,1058,17906239000128,,3548708,635684676111,122020
842,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,,3505708,206165226110,122020


In [42]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,NaN,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
4,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,102019
...,...,...,...,...,...,...,...,...,...
272,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,,3548906,637117955110,122020
273,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,,3549904,645333991113,122020
274,0150,000359053,RESTAURANTE E LANCHONETE D JULIA ME,1058,17906239000128,,3548708,635684676111,122020
275,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,,3505708,206165226110,122020


In [43]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    group['IE'] = non_null_ie.iloc[0] if not non_null_ie.empty else None
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final = df_0150_final.groupby('CNPJ').apply(fill_missing_ie)
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3564766384.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_0150_final = df_0150_final.groupby('CNPJ').apply(fill_missing_ie)


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
0,0150,000000451,Master fórmula farmácia de manipulação ltda - me,1058,71605265000161,,3550308,114031767115,102019
1,0150,000000454,Master formula farmacia de manipulação ltda-me,1058,71605265000404,,3548708,635724845110,102019
2,0150,000000492,Master Formula Farmacia de Manipulacao LTDA,1058,71605265004230,,3550308,126625221115,102019
3,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,,3510500,254233469115,102019
4,0150,000002002,Distribuidora de Medicamentos Santa Cruz Ltda,1058,61940292000137,,3534401,492218057113,102019
...,...,...,...,...,...,...,...,...,...
272,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,,3548906,637117955110,122020
273,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,,3549904,645333991113,122020
274,0150,000359053,RESTAURANTE E LANCHONETE D JULIA ME,1058,17906239000128,,3548708,635684676111,122020
275,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,,3505708,206165226110,122020


In [44]:
df_0150_final.loc[(df_0150_final['CNPJ'].notnull()) & (df_0150_final['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,PERIODO
100,0150,000174315,Instituto Argonauta Para A Conservacao Costeir...,1058,02600247000174,,3551009,None,042020
118,0150,751995449,Instituto central de gastroenterologia do abc,1058,49523806000153,,3548708,None,052020
214,0150,000174315,Instituto Argonauta Para A Conservacao Costeir...,1058,02600247000174,,3555406,None,092020
247,0150,000001180,DAZ Participações LTDA,1058,18901772000160,,3549904,None,112020


In [51]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['PERIODO'] == date]
    
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final[df_0150_final['PERIODO'] == date]
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='COD_ITEM', keep='first', inplace=True)
    
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['COD_ITEM'].unique()
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['COD_ITEM','QTD_INI', 'ICMS_INI', 'QTD_FIM', 'ICMS_FIN']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['QTD_FIM'] = df_1050['QTD_FIM'].astype(float).astype(int)
    df_1050['ICMS_FIN'] = df_1050['ICMS_FIN'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'COD_ITEM', 'QTD_INI', 'ICMS_INI', 'QTD_FIM', 'ICMS_FIN']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'Loja_44/CAT_Loja_44/teste/0000_{date}_v3_1.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'Loja_44/CAT_Loja_44/teste/0150_{date}_v3_1.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'Loja_44/CAT_Loja_44/teste/0200_{date}_v3_1.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'Loja_44/CAT_Loja_44/teste/1050_{date}_v3_1.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'Loja_44/CAT_Loja_44/teste/1100_{date}_v3_1.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"Loja_44/CAT_Loja_44/teste/*{date}_v3_1.txt")
    
    with open(f"Loja_44/CAT_Loja_44/cat_loja_44_{date}_v3_1.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_7448\3820126039.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu